In [1]:
import numpy as np 
import pandas as pd 
from subprocess import check_output
#print(check_output(["ls", "../input/"]))
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from os.path import join as opj
from matplotlib import pyplot as plt
import tensorflow as tf
import os
%matplotlib inline


In [2]:
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, Activation,Add, ZeroPadding2D ,AveragePooling2D
from keras.layers import GlobalMaxPooling2D,concatenate
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Model
from keras import initializers,regularizers
from keras.optimizers import Adam,SGD
from keras.initializers import glorot_uniform
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
import keras.backend as K
K.set_image_data_format('channels_last')
#K.set_learning_phase(1)

Using TensorFlow backend.


In [3]:
train = pd.read_json("input/train.json")

In [4]:
def collect_test():
    test_data = pd.DataFrame()
    for i in range(4):
        test_piece = pd.read_csv('input/test_subparts/test'+str(i+1)+'.csv')
        test_data = pd.concat([test_data,test_piece])
        test_data = test_data.reset_index().drop(['index'],axis = 1)
    return test_data

In [5]:
test_data = collect_test()

In [6]:
def stats_and_fill_na(data,training = True):
    if training ==True:
        data['b1_median'] = [np.median(i) for i in data['band_1']]
        data['b2_median'] = [np.median(i) for i in data['band_2']]
        data['b1_mean'] = [np.mean(i) for i in data['band_1']]
        data['b2_mean'] = [np.mean(i) for i in data['band_2']]
        data['b1_max'] = [np.max(i) for i in data['band_1']]
        data['b2_max'] = [np.max(i) for i in data['band_2']]
        data['b1_min'] = [np.min(i) for i in data['band_1']]
        data['b2_min'] = [np.min(i) for i in data['band_2']]
        data['b1_std'] = [np.std(i) for i in data['band_1']]
        data['b2_std'] = [np.std(i) for i in data['band_2']]
        data['b1_var'] = [np.var(i) for i in data['band_1']]
        data['b2_var'] = [np.var(i) for i in data['band_2']]
        global stats
        stats = data.columns[-12:]
        X_train = data[data['inc_angle']!='na'][stats]
        y_train = data[data['inc_angle']!='na']['inc_angle']
        lr_model = LinearRegression().fit(X_train,y_train)
        data['inc_angle'][data['inc_angle']=='na'] = lr_model.predict(data[stats])
    else:
        only_band1 = ['band1_'+str(i) for i in range(75*75)]
        only_band2 = ['band2_'+str(i) for i in range(75*75)]
        both_bands = only_band1+only_band2
        data['b1_median'] = np.median(data[only_band1],axis = 1)
        data['b2_median'] = np.median(data[only_band2],axis = 1)
        data['b1_mean'] = np.mean(data[only_band1],axis = 1)
        data['b2_mean'] = np.mean(data[only_band2],axis = 1)
        data['b1_max'] = np.max(data[only_band1],axis = 1)
        data['b2_max'] = np.max(data[only_band2],axis = 1)
        data['b1_min'] = np.min(data[only_band1],axis = 1)
        data['b2_min'] = np.min(data[only_band2],axis = 1)
        data['b1_std'] = np.std(data[only_band1],axis = 1)
        data['b2_std'] = np.std(data[only_band2],axis = 1)
        data['b1_var'] = np.var(data[only_band1],axis = 1)
        data['b2_var'] = np.var(data[only_band2],axis = 1)
    return data

In [7]:
train = stats_and_fill_na(train)

c:\users\yltbe\anaconda3\envs\tensorflow_gpu\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [8]:
test_data = stats_and_fill_na(test_data,training = False)

In [9]:
def prepare_band12(data,training = True):
    if training==True:
        X_band_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in data["band_1"]])
        X_band_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in data["band_2"]])
        target_train = data['is_iceberg']
        X_other = X_other = data.drop(['band_1','band_2','id','is_iceberg'],axis = 1)
        return X_band_1,X_band_2,target_train,X_other
    else :
        only_band1 = ['band1_'+str(i) for i in range(75*75)]
        only_band2 = ['band2_'+str(i) for i in range(75*75)]
        both_bands = only_band1+only_band2
        X_band_1=np.array(data[only_band1]).reshape(8424,75,75)
        X_band_2=np.array(data[only_band2]).reshape(8424,75,75)
        X_other = data.iloc[:,-13:]
        return X_band_1,X_band_2,X_other

In [10]:
X_band_1,X_band_2,target,X_others = prepare_band12(train)

In [11]:
X_band_test_1,X_band_test_2,X_test_others = prepare_band12(test_data,training=False)

In [12]:
def prepare_RGB(band1,band2,method = 'third mean'):
    
    if method=='third mean':
        third = (band1+band2)/2
        image = np.concatenate([band1[:, :, :, np.newaxis],
                                band2[:, :, :, np.newaxis], 
                                third[:, :, :, np.newaxis]], axis=-1)
    elif method=='third max':
        third = np.max(np.concatenate([band1[:, :, :, np.newaxis], band2[:, :, :, np.newaxis]],axis = -1),axis = -1)
        image = np.concatenate([band1[:, :, :, np.newaxis],
                                band2[:, :, :, np.newaxis], 
                                third[:, :, :, np.newaxis]], axis=-1)
    return image

In [13]:
X = prepare_RGB(X_band_1,X_band_2,method = 'third mean')

In [14]:
X_test = prepare_RGB(X_band_test_1,X_band_test_2,method = 'third mean')

In [223]:

X_train_cv,X_valid,X_train_others,X_valid_others, y_train_cv, y_valid = train_test_split(X, X_others,target, random_state=2,stratify = target_train, train_size=0.80)

c:\users\yltbe\anaconda3\envs\tensorflow_gpu\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [234]:
# scaler = MinMaxScaler().fit(X_train_others)
# X_train_others = scaler.transform(X_train_others)
# X_valid_others = scaler.transform(X_valid_others)

### Cross Validation Folds

In [15]:
def data_augmentation(X_input,Y_input,batch_size = 32):
    data_aug = ImageDataGenerator(#featurewise_center=True,
                             #featurewise_std_normalization=True,
                             #zca_whitening=True,
                             rotation_range=0,
                             width_shift_range = 0,
                             height_shift_range = 0,
                             zoom_range = 0,
                             data_format = 'channels_last',
                             horizontal_flip = True,
                             vertical_flip = True)#,fill_mode = 'constant',cval = 0)
    data_aug_batches = data_aug.flow(X_input,Y_input,batch_size = batch_size)
    return data_aug_batches

def data_augmentation_with_others(X_input,X_others,Y_input,batch_size = 32):
    data_aug = ImageDataGenerator(#featurewise_center=True,
                             #featurewise_std_normalization=True,
                             #zca_whitening=True,
                             rotation_range=0,
                             width_shift_range = 0,
                             height_shift_range = 0,
                             zoom_range = 0,
                             data_format = 'channels_last',
                             horizontal_flip = True,
                             vertical_flip = True)#,fill_mode = 'constant',cval = 0)
    gen1 = data_aug.flow(X_input,Y_input,batch_size = batch_size,seed = 1990)
    gen2 = data_aug.flow(X_input,X_others,batch_size = batch_size,seed = 1990)
    while True:
            Xi1 = gen1.next()
            Xi2 = gen2.next()
            #Assert arrays are equal - this was for peace of mind, but slows down training
            #np.testing.assert_array_equal(X1i[0],X2i[0])
            yield [Xi1[0], Xi2[1]], Xi1[1]

def data_no_augmentation_with_others(X_input,X_others,Y_input,batch_size = 32):
    data_aug = ImageDataGenerator(#featurewise_center=True,
                             #featurewise_std_normalization=True,
                             #zca_whitening=True,
                             rotation_range=0,
                             width_shift_range = 0,
                             height_shift_range = 0,
                             zoom_range = 0,
                             data_format = 'channels_last',
                             horizontal_flip = False,
                             vertical_flip = False)#,fill_mode = 'constant',cval = 0)
    gen1 = data_aug.flow(X_input,Y_input,batch_size = batch_size,seed = 1990)
    gen2 = data_aug.flow(X_input,X_others,batch_size = batch_size,seed = 1990)
    while True:
            Xi1 = gen1.next()
            Xi2 = gen2.next()
            #Assert arrays are equal - this was for peace of mind, but slows down training
            #np.testing.assert_array_equal(X1i[0],X2i[0])
            yield [Xi1[0], Xi2[1]], Xi1[1]



In [16]:

def cnnmodel():
    X_input = Input(shape = (75,75,3))
    #First composite layer
    X = Conv2D(64,kernel_size = (5,5))(X_input)
    X = Activation('relu')(X)
    X = MaxPooling2D(pool_size=(3,3),strides =(2,2))(X)
    X = Dropout(rate = 0.2)(X)
    
    #Second composite layer
    X = Conv2D(128,kernel_size = (3,3))(X)
    X = Activation('relu')(X)
    X = MaxPooling2D(pool_size=(3,3),strides =(2,2))(X)
    X = Dropout(rate = 0.2)(X)
    
    #Third composite layer
    X = Conv2D(128,kernel_size = (3,3))(X)
    X = Activation('relu')(X)
    X = MaxPooling2D(pool_size=(2,2),strides =(2,2))(X)
    X = Dropout(rate = 0.2)(X)

    #Forth composite layer
    X = Conv2D(64,kernel_size = (3,3))(X)
    X = Activation('relu')(X)
    X = MaxPooling2D(pool_size=(3,3),strides =(2,2))(X)
    X = Dropout(rate = 0.2)(X)

    #Flatten layer
    X = Flatten()(X)
    
    #First dense layer
    X = Dense(256,activation = 'relu')(X)
    X = Dropout(rate = 0.2)(X)
   
    #Second dense layer
    X = Dense(256,activation = 'relu')(X)
    X = Dropout(rate = 0.2)(X)
   
    #Decision layer
    X = Dense(1,activation='sigmoid')(X)
    
    model = Model(inputs=X_input,outputs=X)

    return model

def cnnmodel_with_others():
    input_other = Input(shape=(13,), name="other")
    X_other = Dense(16,activation = 'relu')(input_other)
    X_other = Dense(16,activation = 'relu')(X_other)
    X_input = Input(shape = (75,75,3))
    #First composite layer
    X = Conv2D(64,kernel_size = (5,5))(X_input)
    X = Activation('relu')(X)
    X = MaxPooling2D(pool_size=(3,3),strides =(2,2))(X)
    X = Dropout(rate = 0.2)(X)
    
    #Second composite layer
    X = Conv2D(128,kernel_size = (3,3))(X)
    X = Activation('relu')(X)
    X = MaxPooling2D(pool_size=(3,3),strides =(2,2))(X)
    X = Dropout(rate = 0.2)(X)
    
    #Third composite layer
    X = Conv2D(128,kernel_size = (3,3))(X)
    X = Activation('relu')(X)
    X = MaxPooling2D(pool_size=(2,2),strides =(2,2))(X)
    X = Dropout(rate = 0.2)(X)

    #Forth composite layer
    X = Conv2D(64,kernel_size = (3,3))(X)
    X = Activation('relu')(X)
    X = MaxPooling2D(pool_size=(3,3),strides =(2,2))(X)
    X = Dropout(rate = 0.2)(X)

    #Flatten layer
    X = Flatten()(X)
    X = concatenate([X,X_other])
    
    #First dense layer
    X = Dense(256,activation = 'relu')(X)
    X = Dropout(rate = 0.2)(X)
   
    #Second dense layer
    X = Dense(256,activation = 'relu')(X)
    X = Dropout(rate = 0.2)(X)
   
    #Decision layer
    X = Dense(1,activation='sigmoid')(X)
    
    model = Model(inputs=[X_input,input_other],outputs=X)

    return model

def model_compile(model,lr = 0.0001,decay = 0, freezing_layers = None,weights_path = None):
   
    if weights_path:
        model.load_weights(weights_path)
    else:
        pass
    if freezing_layers:
        for layer in model.layers[:freezing_layers]:
            layer.trainable = False
    else:
        pass
    optimizer = Adam(lr = lr, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=decay)
    #optimizer = SGD(lr, decay=0, momentum=0.9, nesterov=True)
    model.compile(loss='binary_crossentropy',optimizer=optimizer,metrics=['accuracy'])
    model.summary()
    return model

def get_callbacks(filepath = ".model_weights.hdf5", patience=7, monitor = 'val_loss', verbose = 1, save_best_only = True):
    es = EarlyStopping('val_loss', patience=patience, mode="min")
    msave = ModelCheckpoint(filepath, monitor=monitor, verbose = verbose, save_best_only=True)
    return [es,msave]


In [17]:

def normal_fit(model,callbacks,epochs = 100,batch_size = 32,verbose = 1):
    
    result = model.fit(X_train_cv, y_train_cv,
                          batch_size=batch_size,
                          epochs=epochs,
                          verbose=verbose,
                          validation_data=(X_valid, y_valid),
                          callbacks=callbacks)
    return result

def data_augment_fit(model,callbacks,epochs = 100,batch_size = 32,verbose = 1):
    
    result = model.fit_generator(data_augmentation(X_train_cv,y_train_cv,batch_size = batch_size),
                          steps_per_epoch = len(X_train_cv)/batch_size,
                          epochs=epochs,
                          verbose=verbose,
                          validation_data=(X_valid, y_valid),
                          callbacks=callbacks)
    return result

def data_augment_fit_with_others(model,callbacks,epochs = 100,batch_size = 32,verbose = 1,da = False):
    
    if da == False:
        result = model.fit_generator(data_no_augmentation_with_others(X_train_cv,X_train_others,y_train_cv,batch_size = 32),
                          steps_per_epoch = len(X_train_cv)/batch_size,
                          epochs=epochs,
                          verbose=verbose,
                          validation_data=([X_valid,X_valid_others], y_valid),
                          callbacks=callbacks)
    else:
        result = model.fit_generator(data_augmentation_with_others(X_train_cv,X_train_others,y_train_cv,batch_size = 32),
                          steps_per_epoch = len(X_train_cv)/batch_size,
                          epochs=epochs,
                          verbose=verbose,
                          validation_data=([X_valid,X_valid_others], y_valid),
                          callbacks=callbacks)
    return result

In [21]:
Cnnmodel = model_compile(cnnmodel_with_others(),lr = 0.0001,decay = 0, freezing_layers = None,weights_path = None)
callbacks = get_callbacks(filepath = 'model save/Model 11-Advanced CNN with DA, self TL and max layer(cv5)/2018.1.4.hdf5', patience=30, save_best_only = True)
result = data_augment_fit_with_others(Cnnmodel,callbacks=callbacks,verbose = 2)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 75, 75, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 71, 71, 64)   4864        input_1[0][0]                    
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 71, 71, 64)   0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 35, 35, 64)   0           activation_1[0][0]               
__________________________________________________________________________________________________
dropout_1 

NameError: name 'X_train_cv' is not defined

In [ ]:
Cnnmodel = model_compile(cnnmodel_with_others(),lr = 0.0001,decay = 0, freezing_layers = 5,weights_path = 'model save/Model 7-2x filters Advanced CNN with DA and self TL/2018.1.2.hdf5')
callbacks = get_callbacks(filepath = 'model save/Model 7-2x filters Advanced CNN with DA and self TL/2018.1.2_stage2.hdf5', patience=30, save_best_only = True)
result = data_augment_fit(Cnnmodel,callbacks = callbacks,epochs = 100,batch_size=32,verbose=2)

In [ ]:
Cnnmodel.load_weights('model save/Model 7-2x filters Advanced CNN with DA and self TL/Model-7 cv1_stage1.hdf5')
Cnnmodel.evaluate(X_valid, y_valid)
#Cnnmodel.evaluate(X_train_cv, y_train_cv)

In [ ]:
def save_history(result,name = 'fit_history.csv',path = os.getcwd()):
    train_loss = result.history['loss']
    val_loss = result.history['val_loss']
    train_accuracy = result.history['acc']
    val_accuracy = result.history['val_acc']
    fit_history = pd.DataFrame({'train_loss':train_loss,'val_loss':val_loss,'train_accuracy':train_accuracy,'val_accuracy':val_accuracy})
    fit_history.to_csv(os.path.join(path,name))

In [ ]:
save_history(result_Cnn)

### Prediction

In [37]:
Cnnmodel.predict([X_train_cv,X_train_others]).reshape(1283)+0

array([ 0.47316885,  0.43016976,  0.49740523, ...,  0.49190184,
        0.48540697,  0.48072121], dtype=float32)

In [47]:
def Model11_CV(K=3):
    folds = list(StratifiedKFold(n_splits=K, shuffle=True,random_state = 3).split(X, target))
    y_train_pred_log=0
    y_valid_pred_log = 0.0*target
    for j, (train_idx, valid_idx) in enumerate(folds):
        print('\n===================FOLD=',j+1)
        global X_train_cv
        X_train_cv = X[train_idx]
        global y_train_cv
        y_train_cv = target[train_idx]
        global X_valid
        X_valid = X[valid_idx]
        global y_valid
        y_valid= target[valid_idx]
        global X_train_others
        X_train_others = np.array(X_others)[train_idx]
        global X_valid_others
        X_valid_others = np.array(X_others)[valid_idx]
        
        scaler = MinMaxScaler().fit(X_train_others)
        X_train_others = scaler.transform(X_train_others)
        X_valid_others = scaler.transform(X_valid_others)
        #Angle
#         X_angle_cv=X_angle[train_idx]
#         X_angle_hold=X_angle[test_idx]

        #define file path and get callbacks
    
        file_path_stage1 = 'model save/Model 11-Advanced CNN with DA, self TL and max layer(cv5)/Model-11 cv'+str(j+1)+'_stage1.hdf5'
        Cnnmodel = model_compile(cnnmodel_with_others(),lr = 0.0001,decay = 0, freezing_layers = None,weights_path = None)
        callbacks = get_callbacks(filepath = file_path_stage1, patience=30, save_best_only = True)
        result1 = data_augment_fit_with_others(Cnnmodel,callbacks = callbacks,epochs = 150,verbose=2)
        print('CV'+str(j+1)+', Done!')
        
        file_path_stage2 = 'model save/Model 11-Advanced CNN with DA, self TL and max layer(cv5)/Model-11 cv'+str(j+1)+'_stage2.hdf5'
        Cnnmodel = model_compile(cnnmodel_with_others(),lr = 0.0001,decay = 0, freezing_layers = 5,weights_path = file_path_stage1)
        callbacks = get_callbacks(filepath = file_path_stage2, patience=30, save_best_only = True)
        result2 = data_augment_fit_with_others(Cnnmodel,callbacks = callbacks,epochs = 150,verbose=2,da=True)
        print('CV'+str(j+1)+', Done!')
#         callbacks = get_callbacks(filepath=file_path, patience=5)
#         gen_flow = gen_flow_for_two_inputs(X_train_cv, X_angle_cv, y_train_cv)
#         galaxyModel= getVggAngleModel()
#         galaxyModel.fit_generator(
#                 gen_flow,
#                 steps_per_epoch=24,
#                 epochs=100,
#                 shuffle=True,
#                 verbose=1,
#                 validation_data=([X_holdout,X_angle_hold], Y_holdout),
#                 callbacks=callbacks)

        #Getting the Best Model
        Cnnmodel.load_weights(filepath=file_path_stage2)
        #Getting Training Score
        score = Cnnmodel.evaluate([X_train_cv,X_train_others],y_train_cv)                                   
        print('Train loss:', score[0])
        print('Train accuracy:', score[1])
        #Getting Test Score
        score = Cnnmodel.evaluate([X_valid,X_valid_others], y_valid)
        print('Valid loss:', score[0])
        print('Valid accuracy:', score[1])

        #Getting validation Score.
        pred_valid=Cnnmodel.predict([X_valid,X_valid_others])
        y_valid_pred_log[valid_idx] = pred_valid.reshape(pred_valid.shape[0])

        #Getting Test Scores
#         temp_test=Cnnmodel.predict(X_test)
#         y_test_pred_log+=temp_test.reshape(temp_test.shape[0])

        #Getting Train Scores
#         temp_train=Cnnmodel.predict([X_train_cv,X_train_others])
#         y_train_pred_log+=temp_train.reshape(temp_train.shape[0])

#     y_valid_pred_log=y_valid_pred_log/K
    #y_train_pred_log=y_train_pred_log/K

#     print('\n Train Log Loss Validation= ',log_loss(target, y_train_pred_log))
    print(' Test Log Loss Validation= ',log_loss(target, y_valid_pred_log))
    return y_valid_pred_log

In [48]:
whole_valid = Model11_CV(K = 5)


===================FOLD= 1


c:\users\yltbe\anaconda3\envs\tensorflow_gpu\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_16 (InputLayer)           (None, 75, 75, 3)    0                                            
__________________________________________________________________________________________________
conv2d_61 (Conv2D)              (None, 71, 71, 64)   4864        input_16[0][0]                   
__________________________________________________________________________________________________
activation_61 (Activation)      (None, 71, 71, 64)   0           conv2d_61[0][0]                  
__________________________________________________________________________________________________
max_pooling2d_61 (MaxPooling2D) (None, 35, 35, 64)   0           activation_61[0][0]              
__________________________________________________________________________________________________
dropout_91

Epoch 11/150
Epoch 00011: val_loss improved from 0.50535 to 0.45251, saving model to model save/Model 11-Advanced CNN with DA, self TL and max layer(cv5)/Model-11 cv1_stage1.hdf5
 - 6s - loss: 0.4855 - acc: 0.7514 - val_loss: 0.4525 - val_acc: 0.7733
Epoch 12/150
Epoch 00012: val_loss improved from 0.45251 to 0.43462, saving model to model save/Model 11-Advanced CNN with DA, self TL and max layer(cv5)/Model-11 cv1_stage1.hdf5
 - 6s - loss: 0.4472 - acc: 0.7712 - val_loss: 0.4346 - val_acc: 0.7857
Epoch 13/150
Epoch 00013: val_loss improved from 0.43462 to 0.40216, saving model to model save/Model 11-Advanced CNN with DA, self TL and max layer(cv5)/Model-11 cv1_stage1.hdf5
 - 6s - loss: 0.4229 - acc: 0.7895 - val_loss: 0.4022 - val_acc: 0.7981
Epoch 14/150
Epoch 00014: val_loss improved from 0.40216 to 0.37459, saving model to model save/Model 11-Advanced CNN with DA, self TL and max layer(cv5)/Model-11 cv1_stage1.hdf5
 - 6s - loss: 0.4001 - acc: 0.8154 - val_loss: 0.3746 - val_acc: 0.8

Epoch 56/150
Epoch 00056: val_loss improved from 0.26556 to 0.25648, saving model to model save/Model 11-Advanced CNN with DA, self TL and max layer(cv5)/Model-11 cv1_stage1.hdf5
 - 6s - loss: 0.2174 - acc: 0.9088 - val_loss: 0.2565 - val_acc: 0.8882
Epoch 57/150
Epoch 00057: val_loss did not improve
 - 6s - loss: 0.1864 - acc: 0.9237 - val_loss: 0.2641 - val_acc: 0.8851
Epoch 58/150
Epoch 00058: val_loss did not improve
 - 6s - loss: 0.1986 - acc: 0.9222 - val_loss: 0.2620 - val_acc: 0.8851
Epoch 59/150
Epoch 00059: val_loss did not improve
 - 5s - loss: 0.1822 - acc: 0.9222 - val_loss: 0.2701 - val_acc: 0.8851
Epoch 60/150
Epoch 00060: val_loss improved from 0.25648 to 0.25325, saving model to model save/Model 11-Advanced CNN with DA, self TL and max layer(cv5)/Model-11 cv1_stage1.hdf5
 - 6s - loss: 0.1755 - acc: 0.9321 - val_loss: 0.2532 - val_acc: 0.8913
Epoch 61/150
Epoch 00061: val_loss did not improve
 - 6s - loss: 0.1822 - acc: 0.9306 - val_loss: 0.2627 - val_acc: 0.8913
Epoch 

...

Epoch 61/150
Epoch 00061: val_loss did not improve
 - 4s - loss: 0.1899 - acc: 0.9176 - val_loss: 0.2100 - val_acc: 0.8969
Epoch 62/150
Epoch 00062: val_loss did not improve
 - 4s - loss: 0.1757 - acc: 0.9199 - val_loss: 0.2138 - val_acc: 0.9094
Epoch 63/150
Epoch 00063: val_loss did not improve
 - 4s - loss: 0.1780 - acc: 0.9237 - val_loss: 0.2087 - val_acc: 0.9000
Epoch 64/150
Epoch 00064: val_loss did not improve
 - 4s - loss: 0.1788 - acc: 0.9268 - val_loss: 0.2130 - val_acc: 0.9094
Epoch 65/150
Epoch 00065: val_loss did not improve
 - 4s - loss: 0.1958 - acc: 0.9199 - val_loss: 0.2025 - val_acc: 0.9094
Epoch 66/150
Epoch 00066: val_loss did not improve
 - 4s - loss: 0.1623 - acc: 0.9337 - val_loss: 0.2113 - val_acc: 0.9031
Epoch 67/150
Epoch 00067: val_loss did not improve
 - 4s - loss: 0.1597 - acc: 0.9367 - val_loss: 0.2208 - val_acc: 0.9094
Epoch 68/150
Epoch 00068: val_loss did not improve
 - 4s - loss: 0.1708 - acc: 0.9276 - val_loss: 0.2061 - val_acc: 0.9156
Epoch 69/150
Epo

Epoch 1/150
Epoch 00001: val_loss improved from inf to 0.68168, saving model to model save/Model 11-Advanced CNN with DA, self TL and max layer(cv5)/Model-11 cv5_stage1.hdf5
 - 12s - loss: 1.3316 - acc: 0.5229 - val_loss: 0.6817 - val_acc: 0.5969
Epoch 2/150
Epoch 00002: val_loss improved from 0.68168 to 0.65134, saving model to model save/Model 11-Advanced CNN with DA, self TL and max layer(cv5)/Model-11 cv5_stage1.hdf5
 - 5s - loss: 0.7251 - acc: 0.5334 - val_loss: 0.6513 - val_acc: 0.6031
Epoch 3/150
Epoch 00003: val_loss improved from 0.65134 to 0.62043, saving model to model save/Model 11-Advanced CNN with DA, self TL and max layer(cv5)/Model-11 cv5_stage1.hdf5
 - 5s - loss: 0.6710 - acc: 0.5596 - val_loss: 0.6204 - val_acc: 0.6344
Epoch 4/150
Epoch 00004: val_loss improved from 0.62043 to 0.59054, saving model to model save/Model 11-Advanced CNN with DA, self TL and max layer(cv5)/Model-11 cv5_stage1.hdf5
 - 5s - loss: 0.6343 - acc: 0.5905 - val_loss: 0.5905 - val_acc: 0.6469
Epo

Epoch 39/150
Epoch 00039: val_loss did not improve
 - 5s - loss: 0.2279 - acc: 0.9032 - val_loss: 0.2639 - val_acc: 0.8781
Epoch 40/150
Epoch 00040: val_loss did not improve
 - 5s - loss: 0.2249 - acc: 0.9077 - val_loss: 0.3077 - val_acc: 0.8500
Epoch 41/150
Epoch 00041: val_loss did not improve
 - 5s - loss: 0.2393 - acc: 0.8911 - val_loss: 0.3289 - val_acc: 0.8375
Epoch 42/150
Epoch 00042: val_loss did not improve
 - 5s - loss: 0.2298 - acc: 0.8955 - val_loss: 0.3282 - val_acc: 0.8438
Epoch 43/150
Epoch 00043: val_loss did not improve
 - 5s - loss: 0.2208 - acc: 0.9032 - val_loss: 0.3358 - val_acc: 0.8375
Epoch 44/150
Epoch 00044: val_loss did not improve
 - 5s - loss: 0.2533 - acc: 0.8917 - val_loss: 0.2891 - val_acc: 0.8656
Epoch 45/150
Epoch 00045: val_loss did not improve
 - 5s - loss: 0.2094 - acc: 0.9054 - val_loss: 0.2672 - val_acc: 0.8812
Epoch 46/150
Epoch 00046: val_loss improved from 0.25584 to 0.23998, saving model to model save/Model 11-Advanced CNN with DA, self TL and 

Epoch 1/150
Epoch 00001: val_loss improved from inf to 0.25216, saving model to model save/Model 11-Advanced CNN with DA, self TL and max layer(cv5)/Model-11 cv5_stage2.hdf5
 - 10s - loss: 0.3359 - acc: 0.8545 - val_loss: 0.2522 - val_acc: 0.9031
Epoch 2/150
Epoch 00002: val_loss improved from 0.25216 to 0.24290, saving model to model save/Model 11-Advanced CNN with DA, self TL and max layer(cv5)/Model-11 cv5_stage2.hdf5
 - 4s - loss: 0.3180 - acc: 0.8665 - val_loss: 0.2429 - val_acc: 0.9156
Epoch 3/150
Epoch 00003: val_loss did not improve
 - 4s - loss: 0.2871 - acc: 0.8673 - val_loss: 0.2568 - val_acc: 0.9062
Epoch 4/150
Epoch 00004: val_loss improved from 0.24290 to 0.22334, saving model to model save/Model 11-Advanced CNN with DA, self TL and max layer(cv5)/Model-11 cv5_stage2.hdf5
 - 4s - loss: 0.2641 - acc: 0.8818 - val_loss: 0.2233 - val_acc: 0.9156
Epoch 5/150
Epoch 00005: val_loss did not improve
 - 4s - loss: 0.2652 - acc: 0.8864 - val_loss: 0.2270 - val_acc: 0.9156
Epoch 6/1

Epoch 58/150
Epoch 00058: val_loss did not improve
 - 4s - loss: 0.1652 - acc: 0.9352 - val_loss: 0.2025 - val_acc: 0.9313
Epoch 59/150
Epoch 00059: val_loss did not improve
 - 4s - loss: 0.1878 - acc: 0.9215 - val_loss: 0.2115 - val_acc: 0.9281
Epoch 60/150
Epoch 00060: val_loss did not improve
 - 4s - loss: 0.1772 - acc: 0.9199 - val_loss: 0.2300 - val_acc: 0.9219
Epoch 61/150
Epoch 00061: val_loss did not improve
 - 4s - loss: 0.1826 - acc: 0.9216 - val_loss: 0.2142 - val_acc: 0.9281
Epoch 62/150
Epoch 00062: val_loss did not improve
 - 4s - loss: 0.1939 - acc: 0.9269 - val_loss: 0.2163 - val_acc: 0.9250
Epoch 63/150
Epoch 00063: val_loss did not improve
 - 4s - loss: 0.1773 - acc: 0.9283 - val_loss: 0.2286 - val_acc: 0.9187
Epoch 64/150
Epoch 00064: val_loss did not improve
 - 4s - loss: 0.1686 - acc: 0.9321 - val_loss: 0.2162 - val_acc: 0.9219
CV5, Done!
1284/1284 [==============================] - 2s 1ms/step
Train loss: 0.153598635135
Train accuracy: 0.939252336449
320/320 [====

### Submission

In [ ]:
def Evaluation_CV(X_train,K=3):
    folds = list(StratifiedKFold(n_splits=K, shuffle=True,random_state = 3).split(X_train, target_train))
    y_test_pred_log=0
    y_train_pred_log=0
    y_valid_pred_log = 0.0*target_train
    for j, (train_idx, valid_idx) in enumerate(folds):
        print('\n===================FOLD=',j+1)
        X_train_cv = X_train[train_idx]
        y_train_cv = target_train[train_idx]
        X_valid = X_train[valid_idx]
        y_valid= target_train[valid_idx]
        
        #Angle
#         X_angle_cv=X_angle[train_idx]
#         X_angle_hold=X_angle[test_idx]

        #define file path and get callbacks
        file_path = 'model save/Model 6-Advanced CNN with DA and self TL(cv3)/Model-6 cv'+str(j+1)+'.hdf5'
        Cnnmodel=cnnmodel(input_shape = (75,75,3),lr = 0.0001)
        Cnnmodel.load_weights(file_path)


#         callbacks = get_callbacks(filepath=file_path, patience=5)
#         gen_flow = gen_flow_for_two_inputs(X_train_cv, X_angle_cv, y_train_cv)
#         galaxyModel= getVggAngleModel()
#         galaxyModel.fit_generator(
#                 gen_flow,
#                 steps_per_epoch=24,
#                 epochs=100,
#                 shuffle=True,
#                 verbose=1,
#                 validation_data=([X_holdout,X_angle_hold], Y_holdout),
#                 callbacks=callbacks)

        #Getting the Best Model
        #Getting Training Score
        print('cv'+str(j+1))
        score = Cnnmodel.evaluate(X_train_cv, y_train_cv)
        print('Train loss:', score[0])
        print('Train accuracy:', score[1])
        #Getting Test Score
        score = Cnnmodel.evaluate(X_valid, y_valid)
        print('Valid loss:', score[0])
        print('Valid accuracy:', score[1])
        print('\n')

        #Getting validation Score.
        pred_valid=Cnnmodel.predict(X_valid)
        y_valid_pred_log[valid_idx] = pred_valid.reshape(pred_valid.shape[0])

        #Getting Test Scores
        temp_test=Cnnmodel.predict(X_test)
        y_test_pred_log+=temp_test.reshape(temp_test.shape[0])

        #Getting Train Scores
        temp_train=Cnnmodel.predict(X_train)
        y_train_pred_log+=temp_train.reshape(temp_train.shape[0])

    y_valid_pred_log=y_valid_pred_log/K
    y_train_pred_log=y_train_pred_log/K

    print('\n Train Log Loss Validation= ',log_loss(target_train, y_train_pred_log))
    print(' Test Log Loss Validation= ',log_loss(target_train, y_valid_pred_log))
    return y_test_pred_log

In [ ]:
preds = Evaluation_CV(X_train,K=5)
sub = test[['id']]
sub['is_iceberg'] = preds
sub.to_csv('5cv.csv',index = False)

### Submission(min max median stacking)

In [18]:
def test_scores(K=5):
    y_test_pred_log=0
    test_df = test_data[['id']]
    for j in range(K):
        print('\n===================FOLD=',j+1)
        file_path = 'model save/Model 11-Advanced CNN with DA, self TL and max layer(cv5)/Model-11 cv'+str(j+1)+'.hdf5'
        Cnnmodel = model_compile(cnnmodel(),lr = 0.0001,decay = 0, freezing_layers = None,weights_path = file_path)

        #Getting Test Scores
        temp_test=Cnnmodel.predict(X_test)
        test_df['cv'+str(j+1)]=temp_test.reshape(temp_test.shape[0])
    return test_df

def test_scores_scaled_others(X_test_others,K = 5):
    folds = list(StratifiedKFold(n_splits=K, shuffle=True,random_state = 3).split(X, target))
    test_df = test_data[['id']]
    for j, (train_idx, valid_idx) in enumerate(folds):
        print('\n===================FOLD=',j+1)
        
        global X_train_others
        X_train_others = np.array(X_others)[train_idx]

        
        scaler = MinMaxScaler().fit(X_train_others)
        X_test_others = scaler.transform(X_test_others)

        file_path = 'model save/Model 11-Advanced CNN with DA, self TL and max layer(cv5)/Model-11 cv'+str(j+1)+'_stage2.hdf5'
        Cnnmodel = model_compile(cnnmodel_with_others(),lr = 0.0001,decay = 0, freezing_layers = 5,weights_path = file_path)
        temp_test=Cnnmodel.predict([X_test,X_test_others])
        test_df['cv'+str(j+1)]=temp_test.reshape(temp_test.shape[0])
    return test_df


In [19]:
test_df = test_scores_scaled_others(X_test_others = X_test_others)


===================FOLD= 1


c:\users\yltbe\anaconda3\envs\tensorflow_gpu\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 75, 75, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 71, 71, 64)   4864        input_1[0][0]                    
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 71, 71, 64)   0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 35, 35, 64)   0           activation_1[0][0]               
__________________________________________________________________________________________________
dropout_1 

c:\users\yltbe\anaconda3\envs\tensorflow_gpu\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



===================FOLD= 2
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 75, 75, 3)    0                                            
__________________________________________________________________________________________________
conv2d_5 (Conv2D)               (None, 71, 71, 64)   4864        input_2[0][0]                    
__________________________________________________________________________________________________
activation_5 (Activation)       (None, 71, 71, 64)   0           conv2d_5[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_5 (MaxPooling2D)  (None, 35, 35, 64)   0           activation_5[0][0]               
_________________________________________________________________________________


===================FOLD= 4
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 75, 75, 3)    0                                            
__________________________________________________________________________________________________
conv2d_13 (Conv2D)              (None, 71, 71, 64)   4864        input_4[0][0]                    
__________________________________________________________________________________________________
activation_13 (Activation)      (None, 71, 71, 64)   0           conv2d_13[0][0]                  
__________________________________________________________________________________________________
max_pooling2d_13 (MaxPooling2D) (None, 35, 35, 64)   0           activation_13[0][0]              
_________________________________________________________________________________

In [20]:
test_df

,id,cv1,cv2,cv3,cv4,cv5
0,5941774d,5.667688e-02,5.530631e-02,8.635153e-03,1.127893e-03,4.170072e-03
1,4023181e,5.026714e-01,8.580565e-01,3.927149e-01,3.979664e-01,2.290074e-02
2,b20200e4,2.789924e-06,3.951226e-10,1.032073e-10,5.433600e-08,8.329140e-08
3,e7f018bb,9.994588e-01,9.999734e-01,9.998581e-01,9.999951e-01,9.999849e-01
4,4371c8c3,1.669144e-02,6.438503e-02,4.461252e-02,3.018984e-03,1.088171e-04
5,a8d9b1fd,1.406482e-01,1.800535e-02,2.911703e-03,8.031805e-03,2.462742e-03
6,29e7727e,1.855465e-01,1.123179e-01,7.316975e-02,5.049995e-03,2.020663e-02
7,92a51ffb,9.979691e-01,9.999943e-01,9.998738e-01,9.999973e-01,9.999726e-01
8,c769ac97,8.177361e-05,5.616964e-05,2.393075e-06,2.250952e-06,6.520145e-07
9,aee0547d,4.983280e-07,2.763404e-07,2.184263e-08,1.156752e-09,2.064336e-09


In [21]:
def get_sub_mmm(test_df):
    prediction = test_df
    prediction['is_iceberg'] = np.where(np.all(prediction[prediction.columns[1:]]>0.9,axis = 1),
                                    np.max(prediction[prediction.columns[1:]],axis = 1),
                                     np.where(np.all(prediction[prediction.columns[1:]]<0.1,axis = 1),np.min(prediction[prediction.columns[1:]],axis = 1),np.median(prediction[prediction.columns[1:]],axis = 1)))
    sub = prediction[['id','is_iceberg']]
    sub.to_csv('sub.csv',index = False)
    return sub

In [22]:
sub = get_sub_mmm(test_df)

c:\users\yltbe\anaconda3\envs\tensorflow_gpu\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


### Multiple submission stacking

In [28]:
path = 'prediction/stacking 1/'

In [34]:
sublist = os.listdir(path)
test_df = test_data[['id']]
for i in sublist:
    temp = pd.read_csv(os.path.join(path,i))
    test_df[i] = temp['is_iceberg']
test_df

c:\users\yltbe\anaconda3\envs\tensorflow_gpu\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


,id,lb0.1661.csv,lb0.1717.csv,lb0.1728.csv,lb0.1978.csv,lb0.2168.csv,mean-(2)(4)lb0.1717.csv
0,5941774d,4.883857e-02,3.457386e-02,7.377190e-03,3.609517e-03,2.721015e-03,8.815090e-02
1,4023181e,3.974485e-01,4.705284e-01,4.416436e-01,3.767341e-01,4.024012e-01,4.446322e-01
2,b20200e4,4.541493e-12,2.426515e-14,5.958568e-11,7.925581e-10,6.370236e-12,3.539300e-10
3,e7f018bb,9.999995e-01,9.999987e-01,9.999954e-01,9.999983e-01,9.999939e-01,9.999878e-01
4,4371c8c3,1.377228e-04,7.653040e-05,4.422425e-04,3.091512e-03,2.120113e-04,3.209088e-03
5,a8d9b1fd,4.990788e-02,8.502229e-02,5.834405e-02,6.308465e-02,7.421116e-04,2.790584e-02
6,29e7727e,1.211617e-01,1.527639e-01,1.079694e-01,1.907021e-01,1.918441e-02,1.444582e-02
7,92a51ffb,9.999950e-01,9.999959e-01,9.999992e-01,9.999897e-01,9.999989e-01,9.999560e-01
8,c769ac97,2.814740e-12,1.046281e-08,2.635662e-09,2.700044e-09,3.130832e-06,3.906417e-09
9,aee0547d,2.542710e-13,6.446605e-12,1.078889e-14,3.189674e-13,1.297212e-08,4.328927e-10


In [40]:
def get_sub_mmm(test_df):
    prediction = test_df
    prediction['is_iceberg'] = np.where(np.all(prediction[prediction.columns[1:]]>0.5,axis = 1),
                                    np.max(prediction[prediction.columns[1:]],axis = 1),
                                     np.where(np.all(prediction[prediction.columns[1:]]<0.5,axis = 1),np.min(prediction[prediction.columns[1:]],axis = 1),prediction['lb0.1661.csv']))
    sub = prediction[['id','is_iceberg']]
    sub.to_csv('sub.csv',index = False)
    return sub

In [41]:
sub = get_sub_mmm(test_df)

c:\users\yltbe\anaconda3\envs\tensorflow_gpu\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [39]:
sub

,id,is_iceberg
0,5941774d,2.721015e-03
1,4023181e,3.974485e-01
2,b20200e4,2.426515e-14
3,e7f018bb,9.999995e-01
4,4371c8c3,7.653040e-05
5,a8d9b1fd,7.421116e-04
6,29e7727e,1.444582e-02
7,92a51ffb,9.999992e-01
8,c769ac97,2.814740e-12
9,aee0547d,1.078889e-14
